# Actividad 4.2
## Ejercicio de Detección de Señales de Tránsito

---

### Integrantes del Equipo 23:
* Carlos Pano Hernández - A01066264 [Campus Estado de México]
* Marie Kate Palau - A01705711 [Campus Monterrey]
* Edson Ulises Rodríguez Dávalos - A01796057 [Campus CdMx]
* Yohanna Ceballos Salomón - A01795115 [Campus Monterrey]

---

### Escuela de Ingeniería y Ciencias, Tecnológico de Monterrey
**Navegación autónoma (MR4010 - Gpo 10)**

---

#### Profesor Titular:
Dr. David Antonio Torres

#### Profesor Asistente:
Mtra. María Mylen Treviño Elizondo

---

**Sábado 07 de junio del 2025**

# 1. Library Imports and Setup
This section contains all the necessary library imports for data manipulation, image processing, deep learning, and visualization that we'll use throughout this notebook.

In [29]:
# OS and file handling
import os
import glob
from PIL import Image

# Data manipulation and numerical computations
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler

# Image processing
import cv2
from skimage.feature import hog

# Data visualization
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

# Deep learning with TensorFlow/Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from skimage.feature import hog
from tensorflow.keras.utils import plot_model

# Additional Keras imports
import keras
from keras.optimizers import Adam

# Configure matplotlib for Jupyter
%matplotlib inline

# 2. Data Loading and Preprocessing
This section handles loading the training and test data from CSV files, extracting key image dimensions, and setting up important constants that will be used throughout the model training process.

In [ ]:
# Load the CSV files
train_df = pd.read_csv('data/Train.csv')
test_df = pd.read_csv('data/Test.csv')

print(train_df.head())
print('*'*100)
print(test_df.head())

# Get image dimensions from training data
IMG_WIDTH = train_df['Width'].median()
IMG_HEIGHT = train_df['Height'].median()
NUM_CHANNELS = 3  # RGB images
NUM_CLASSES = len(train_df['ClassId'].unique())

print(f"Image dimensions: {IMG_WIDTH}x{IMG_HEIGHT}")

# 3. Loading and Preprocessing Images
The following code block loads and preprocesses the traffic sign images from the file paths specified in our training and test datasets. Each image is:
- Loaded from disk.
- Resized to consistent dimensions ({IMG_WIDTH}x{IMG_HEIGHT}).
- Normalized to [0,1] range.
- Converted to numpy arrays for model training.

In [ ]:
# Function to load and preprocess images
def load_and_preprocess_image(file_path):
    # Load image
    img = Image.open(file_path)
    # Resize to median dimensions
    img = img.resize((int(IMG_WIDTH), int(IMG_HEIGHT)))
    # Convert to numpy array and normalize
    img_array = np.array(img) / 255.0
    return img_array

# Prepare training data -> For this exercise we are using the data from the Train.csv file under Data folder
X_train = np.array([load_and_preprocess_image(os.path.join('data', path)) for path in train_df['Path']])
y_train = train_df['ClassId'].values

# Prepare test data -> For this exercise we are using the data from the Test.csv file under Data folder
X_test = np.array([load_and_preprocess_image(os.path.join('data', path)) for path in test_df['Path']])
y_test = test_df['ClassId'].values

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

# 4. Loading and Preprocessing Images
The following code block loads and preprocesses the traffic sign images from the file paths specified in our training and test datasets. Each image is:
- Loaded from disk.
- Resized to consistent dimensions ({IMG_WIDTH}x{IMG_HEIGHT}).
- Normalized to [0,1] range.
- Converted to numpy arrays for model training.

In [ ]:
n_muestras = []
n_digitos = 10

for n in range(n_digitos):
    x_sel = X_train[y_train == n]
    n_muestras.append(len(x_sel))
plt.figure(figsize=(8,6))
plt.bar(range(0,n_digitos), n_muestras)
plt.xlabel("Dígito")
plt.ylabel("Número de Muestras")
plt.show()

# 4. Model Architecture

The following code block defines and compiles our CNN model architecture for traffic sign classification. The model consists of:
- Three convolutional blocks with increasing filter sizes (32->64->128).
- Each block has batch normalization, max pooling and dropout for regularization.
- Dense layers for final classification across 43 traffic sign categories.

In [ ]:
# Create CNN model
model = Sequential([
    # First Convolutional Block
    keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', 
                       input_shape=(int(IMG_HEIGHT), int(IMG_WIDTH), NUM_CHANNELS)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Dropout(0.25),
    
    # Second Convolutional Block
    keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Dropout(0.25),
    
    # Third Convolutional Block
    keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Dropout(0.25),
    
    # Flatten and Dense Layers
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
print(model.summary())

# Generate and save model architecture diagram
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)

# Display the architecture diagram
plt.figure(figsize=(10,10))
img = plt.imread('model_architecture.png')
plt.imshow(img)
plt.axis('off') # Hide axes
plt.show()


# 5. Model Training and Evaluation
In this section, we train the CNN model on our traffic sign dataset and evaluate its performance. We use categorical crossentropy loss, the Adam optimizer, and track both accuracy and loss metrics during training.

In [ ]:
history = model.fit(
    X_train, 
    tf.keras.utils.to_categorical(y_train, NUM_CLASSES),
    validation_split=0.5,
    epochs=10,
    batch_size=400,
    verbose=1,
    shuffle=True
)

scores = model.evaluate(X_test, tf.keras.utils.to_categorical(y_test, NUM_CLASSES), verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# 6. Visualizing Training Results
The following section creates plots to visualize the model's training progress, showing both the loss and accuracy metrics over epochs for training and validation data.

In [ ]:
# Create figure with 2 subplots
plt.figure(figsize=(12,5))

# Plot training & validation loss
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Plot training & validation accuracy 
plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# 7. Testing with Multiple Sample Images
The following section demonstrates how to use the trained model to classify multiple traffic sign images. We load, preprocess, and predict on test images from the Meta folder to verify the model's performance on real-world data. The images are displayed in a grid format with their predicted sign types.

In [ ]:
test_image_path = 'data/Meta/'
test_images = glob.glob(os.path.join(test_image_path, '*.png'))

# Create multiple figures if more than 16 images
num_images = len(test_images)
num_figures = (num_images - 1) // 16 + 1

for fig_num in range(num_figures):
    plt.figure(figsize=(15, 15))
    start_idx = fig_num * 16
    end_idx = min(start_idx + 16, num_images)
    
    for idx, image_path in enumerate(test_images[start_idx:end_idx]):
        # Load and preprocess image
        img = Image.open(image_path)
        img_arr = np.asarray(img)
        img_rs = cv2.resize(img_arr, (43, 43))
        img_gray = cv2.cvtColor(img_rs, cv2.COLOR_BGR2GRAY)
        img_not = cv2.bitwise_not(img_gray)
        
        # Display image
        plt.subplot(4, 4, idx+1)
        plt.imshow(img_not, cmap=plt.get_cmap('gray'))
        plt.axis('off')
        
        # Prepare for prediction
        img_not = img_not/255
        img_resh = img_not.reshape(1, 43, 43, 1)
        img_resh = np.repeat(img_resh, 3, axis=-1)
        
        # Make prediction
        prediction = np.argmax(model.predict(img_resh, verbose=0), axis=-1)
        plt.title(f'Sign Type: {prediction[0]}')

    plt.tight_layout()
    plt.show()